In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx


In [8]:
!pip install pdfminer.six


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.6 MB/s eta 0:00:00


In [11]:
import re
from pdfminer.high_level import extract_text

text = extract_text("leave-policy.pdf")

# Save the text to a file
with open("leave_policy_text.txt", "w", encoding="utf-8") as f:
  f.write(text)


In [22]:
def clean_text(text):
    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove special characters and single-word lines
    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        line = re.sub(r'[^\w\s]', '', line)  # Remove special characters
        if len(line.strip().split()) > 5:    # Keep lines with more than one word
            cleaned_lines.append(line.strip())

    # Remove empty lines
    cleaned_text = '\n'.join(filter(None, cleaned_lines))

    return cleaned_text

# Read the text from the file
with open("leave_policy_text.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Clean the text
cleaned_text = clean_text(text)

# Save the cleaned text to a file
with open("cleaned_leave_policy_text.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_text)


In [37]:
import nltk
import networkx as nx
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import re

nltk.download("punkt")

def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.read()
    file.close()
    return filedata



Summarized text has been written to /content/summarized_output.txt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)



In [ ]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: # Ignore if both are same sentences
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix



In [ ]:
def generate_summary(file_name, output_file, top_n=50):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text and split it into sentences
    article = read_article(file_name)
    sentences = nltk.sent_tokenize(article)

    # Step 2 - Generate Similarity Matrix across sentences
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity matrix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    if top_n > len(ranked_sentence):
        top_n = len(ranked_sentence)

    for i in range(top_n):
        summarize_text.append("".join(ranked_sentence[i][1]))

    # Step 5 - Write the summarized text to the output file
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(". ".join(summarize_text))

    print(f"Summarized text has been written to {output_file}")



In [ ]:

generate_summary("/content/cleaned_leave_policy_text.txt", "/content/summarized_output.txt", 5)